In [5]:
import pandas as pd
import numpy as np
np.int=int
np.bool=bool
np.object=object
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Bidirectional, SpatialDropout1D
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv(r"C:\Users\smura\Downloads\cyberbullying_tweets.csv")

In [3]:
texts = df['tweet_text'].values
labels = df['cyberbullying_type'].values

# Encode labels to numerical values
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Tokenize and pad sequences
vocab_size = 10000  # Limit vocabulary size
max_length = 100  # Maximum length of sequences
trunc_type = 'post'
padding_type = 'post'
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, encoded_labels, test_size=0.2, random_state=42)

In [27]:
model = tf.keras.Sequential([
    Embedding(vocab_size, 128, input_length=max_length),
    SpatialDropout1D(0.3),  # Regularization layer

    # 1D Convolutional layer to extract n-gram features
    Conv1D(filters=128, kernel_size=5, activation='relu'),
    MaxPooling1D(pool_size=2),

    # LSTM layers for sequence learning
    Bidirectional(LSTM(64, return_sequences=True)),
    LSTM(32),

    # Dense layers for classification
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

In [28]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [30]:
#early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model
model.fit(X_train, y_train, epochs=15, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/15
1193/1193 [==============================] - 15s 12ms/step - loss: 0.4600 - accuracy: 0.7897 - val_loss: 0.4969 - val_accuracy: 0.7776
Epoch 2/15
1193/1193 [==============================] - 14s 12ms/step - loss: 0.4239 - accuracy: 0.8005 - val_loss: 0.5235 - val_accuracy: 0.7768
Epoch 3/15
1193/1193 [==============================] - 14s 12ms/step - loss: 0.4004 - accuracy: 0.8044 - val_loss: 0.5246 - val_accuracy: 0.7814
Epoch 4/15
1193/1193 [==============================] - 15s 13ms/step - loss: 0.3783 - accuracy: 0.8103 - val_loss: 0.5233 - val_accuracy: 0.7801
Epoch 5/15
1193/1193 [==============================] - 15s 13ms/step - loss: 0.3635 - accuracy: 0.8177 - val_loss: 0.5221 - val_accuracy: 0.7836
Epoch 6/15
1193/1193 [==============================] - 15s 13ms/step - loss: 0.3402 - accuracy: 0.8364 - val_loss: 0.5497 - val_accuracy: 0.7970
Epoch 7/15
1193/1193 [==============================] - 15s 13ms/step - loss: 0.3170 - accuracy: 0.8611 - val_loss: 0.5523 -

In [31]:
model.save("cyber.h5")

In [6]:
model=load_model("cyber.h5")


In [9]:
from sklearn.metrics import precision_score, recall_score, f1_score
precision = precision_score(y_test,  y_pred_classes, average='micro')
recall = recall_score(y_test,  y_pred_classes, average='micro')
f1 = f1_score(y_test,  y_pred_classes, average='micro')
print(f"Precision Score: {precision * 100:.2f}%")
print(f"Recall Score: {recall * 100:.2f}%")
print(f"F1 Score: {f1 * 100:.2f}%")

Precision Score: 82.33%
Recall Score: 82.33%
F1 Score: 82.33%
